# 실전 문제 해결 (과적합)

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [6]:
class ConvLNReluBock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvLNReluBock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)
        self.ln = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        x = self.ln(x)
        return tf.nn.relu(x)

# Define network architecture
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = ConvLNReluBock(16, (3, 3))
        self.conv1_2 = ConvLNReluBock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv2_1 = ConvLNReluBock(32, (3, 3))
        self.conv2_2 = ConvLNReluBock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))

        self.conv3_1 = ConvLNReluBock(64, (3, 3))
        self.conv3_2 = ConvLNReluBock(64, (3, 3))

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu', 
                                            kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax',
                                            kernel_regularizer=tf.keras.regularizers.l2(0.01))

    def call(self, x, training=False, mask=None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비


In [4]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(1024)

170500096/170498071 [==============================] - 697s 4us/step


## Keras API 모델 학습

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1563/1563 [==============================] - 228s 146ms/step - loss: 2.1967 - accuracy: 0.3981 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
1563/1563 [==============================] - 240s 154ms/step - loss: 1.2805 - accuracy: 0.5889 - val_loss: 1.2031 - val_accuracy: 0.6180
Epoch 3/100
1563/1563 [==============================] - 255s 163ms/step - loss: 1.0841 - accuracy: 0.6605 - val_loss: 1.0222 - val_accuracy: 0.6761
Epoch 4/100
1563/1563 [==============================] - 255s 163ms/step - loss: 0.9772 - accuracy: 0.6989 - val_loss: 1.0243 - val_accuracy: 0.6797
Epoch 5/100
1563/1563 [==============================] - 255s 163ms/step - loss: 0.9202 - accuracy: 0.7181 - val_loss: 0.9965 - val_accuracy: 0.6937
Epoch 6/100
1563/1563 [==============================] - 236s 151ms/step - loss: 0.8703 - accuracy: 0.7389 - val_loss: 0.9061 - val_accuracy: 0.7222
Epoch 7/100
1563/1563 [==============================] - 240s 154ms/step - loss: 0.8249 - accuracy

1563/1563 [==============================] - 209s 134ms/step - loss: 0.2812 - accuracy: 0.9477 - val_loss: 0.9376 - val_accuracy: 0.7744
Epoch 56/100
1563/1563 [==============================] - 210s 134ms/step - loss: 0.2811 - accuracy: 0.9481 - val_loss: 0.9463 - val_accuracy: 0.7828
Epoch 57/100
1563/1563 [==============================] - 210s 135ms/step - loss: 0.2793 - accuracy: 0.9477 - val_loss: 0.9572 - val_accuracy: 0.7727
Epoch 58/100
1563/1563 [==============================] - 237s 152ms/step - loss: 0.2702 - accuracy: 0.9516 - val_loss: 0.9663 - val_accuracy: 0.7803
Epoch 59/100
1563/1563 [==============================] - 233s 149ms/step - loss: 0.2716 - accuracy: 0.9501 - val_loss: 0.9763 - val_accuracy: 0.7797
Epoch 60/100
1563/1563 [==============================] - 234s 150ms/step - loss: 0.2718 - accuracy: 0.9515 - val_loss: 0.9659 - val_accuracy: 0.7756
Epoch 61/100
1563/1563 [==============================] - 235s 151ms/step - loss: 0.2658 - accuracy: 0.9528 - val